In [81]:
import numpy as np

In [89]:
states = list((x, y) for x in [0, 1] for y in [0, 1])
actions = ["ms", "nms"] # ms: mastered skills, nms: not mastered skills
ACTION_PROB = 0.5 # TODO: still not sure what is the ACTION_PROB
# STATE_PROB = 0.5 # TODO: have to figure out what is exactly the STATE_PROB
reward_ms = 1
reward_nms = 10

In [83]:
states

[(0, 0), (0, 1), (1, 0), (1, 1)]

In [84]:
def start_state(state):
    x, y = state
    if x == 0 and y == 0:
        return True

In [85]:
def acquire_one_skill(state):
    x, y = state
    if (x == 0 and y == 1) or (x == 1 and y == 0):
        return True

In [86]:
def is_terminal(state):
    x, y = state
    if (x == 1 and y == 1):
        return True

In [87]:
def step(state, action):
    if action == "ms":
        reward = reward_ms
        next_states = [(state)]
    else:
        if start_state(state):
            next_states = [(0, 1), (1, 0)]
            reward = reward_nms
        elif acquire_one_skill(state):
            next_states = [(1, 1)]
            reward = reward_nms
    return next_states, reward
        

In [88]:
def expected_returns():
    state_value = np.zeros((2, 2))
    new_state_value = state_value.copy()
    
    iteration = 1
    
    state_value_iterations = []
    state_value_iterations.append(state_value)
    while True:
        for i, j in states:
            if is_terminal([i, j]):
                continue
            new_state_value[i, j] = 0
            
            for action in actions:
                next_states, reward = step((i, j), action)
                for next_i, next_j in next_states:
                    new_state_value[i, j] += ACTION_PROB*(1/len(next_states))*(reward + new_state_value[next_i, next_j])
        
        if np.sum(np.abs(new_state_value - state_value)) < 1e-4:
            state_value = new_state_value.copy()
            break
        
        state_value = new_state_value.copy()
        state_value_iterations.append(state_value)
        iteration += 1
    
    return state_value, iteration, state_value_iterations               

In [80]:
state_value, iteration, state_value_iterations = expected_returns()
state_value

array([[ 8.25,  5.5 ],
       [ 5.5 ,  0.  ]])